# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Исследование надёжности заёмщиков
Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.
**Цель исследования** — ответить на вопросы:

1. Есть ли зависимость между наличием детей и возвратом кредита в срок?
2. Есть ли зависимость между семейным положением и возвратом кредита в срок?
3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
4. Как разные цели кредита влияют на его возврат в срок?

**Ход исследования**

Данные о о заемщиках мы получили из файла '/datasets/data.csv'. О качестве данных ничего не известно. Поэтому перед проверкой гипотез понадобится обзор данных.

Таким образом, исследование пройдёт в три этапа:

1. [Обзор данных.](#start)
2. [Предобработка данных.](#preprocessing)
3. [Ответы на вопросы.](#questions)

In [13]:
import pandas as pd
from pymystem3 import Mystem
m = Mystem()
df = pd.read_csv('/datasets/data.csv')
df.info() #посмотрим информацию
df.head() #визуально изучим таблицу

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


**Вывод**

Со столбцами days_employed и total_income явно что-то не так. В total_income тип данных нужно переименовать в int для удобного формата. Так же days_employed часто отрицательные, а в столбце education проблемы с регистром.

## Шаг 2. Предобработка данных

### Обработка пропусков

In [14]:
df.info() # Проверяем типы данных и длины столбцов, для заполнения пропусков 
# в столбцах days_employed и total_income меньше значений, чем в остальных => думаем как заполнить пропуски

income_grouped_mean = df.groupby('income_type')['total_income'].mean() 
# print(income_grouped_mean)

income_grouped_median = df.groupby('income_type')['total_income'].median()
# print(income_grouped_median) # данные среднего и медианы совпадают, заполняем пропуски

def fillbygroup(df, row): # функция заменяет NaN в row на медианное значение этого столбца у соответствующего income_type
    unique_ins_type = df['income_type'].unique()
    for type in unique_ins_type:
        df.loc[df['income_type'] == type, row] = df.loc[df['income_type'] == type, row].fillna(df.loc[df['income_type'] == type]['total_income'].median())
    return df
df = fillbygroup(df, 'total_income') # вызываем df.info() и проверяем замену пропусков 'total_income'
days_employed_mean = df['days_employed'].mean() #найдем среднее для столбца
#заменим отрицательные числа на положительные в стаже
def negative_to_pozitive(value):
    if value < 0:
        value *= -1
        return value
    else:
        return value
df['days_employed'] = df['days_employed'].apply(negative_to_pozitive) #применение метода к значения столбца
df['days_employed'] = df['days_employed'].fillna(days_employed_mean) #заполним пропущенные значения для столбца 'days_employed'
df.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [15]:
df['days_employed'].abs()

0          8437.673028
1          4024.803754
2          5623.422610
3          4124.747207
4        340266.072047
             ...      
21520      4529.316663
21521    343937.404131
21522      2113.346888
21523      3112.481705
21524      1984.507589
Name: days_employed, Length: 21525, dtype: float64

**Вывод**

В столбце days_employed отрицательные числа сделали положительными. заменили пропуски в столбцах days_employed и total_income на среднее значение.

### Замена типа данных

In [16]:
df['days_employed'] = df['days_employed'].astype('int')
df['total_income'] = df['total_income'].astype('int')
#проверяем тип данных через df.info(), визуализируем через df.head(). типы данных заменили)

**Вывод**

В стаже и доходе тип данных стоял float64, теперь заменили на int с помощью функцией astype.

### Обработка дубликатов

In [17]:
from collections import Counter #приведем в единный формат, уберем дубликаты, которые были созданы из-за разного регистра
Counter(['df.education'])
df['education'] = df['education'].str.lower() #приведем строки к нижнему регистру
Counter(['df.education'])
Counter(df['family_status']) #проверяем формат столбца 'семейный статус'
Counter(df['income_type']) # проверяем формат столбца 'профессии'
Counter(df['purpose']) #проверяем формат столбца цель кредита
df['children'].value_counts() #проверим формат столбца количества детей
#видим ошибку в значениях количества детей, заменим -1 на 1, а 20 медианным значением.
df['children'] = df['children'].replace(-1, 1)
children_median = df.loc[df.loc[:, 'children'] != 20]['children'].median()
df['children'] = df['children'].replace(20, children_median)
pd.DataFrame({'Кол-во заёмщиков по кол-ву детей':df['children'].value_counts()})

,Кол-во заёмщиков по кол-ву детей
0,14225
1,4865
2,2055
3,330
4,41
5,9


In [18]:
df.duplicated().sum() #посмотрим на них
df[df.duplicated(keep=False)].sort_values(by=['total_income', 'days_employed']) #видим, что это банальное задвоение, все дубликаты одинаковые. удаляем их
df = df.drop_duplicates() #удаляем
df.duplicated().sum() #проверяем



0

**Вывод**

Привели значения в единный формат, проверили остальные столбцы через метод Counter и метод value_counts(). Были отрицательные значения детей и неверное значение 20 детей. Обработали эти значения.

### Лемматизация

In [19]:
#столбец цель кредита нужно лемматизировать и на основе его создать новый столбец с категориями 

estate = ['недвижим', 'жиль']
wedding = ['свадьб']
auto = ['автомоб']
education = ['образова']

df['purpose_category'] = 0
def categoryzer(list_of_words, category):
    join = '|'.join(list_of_words)
    index = df[df['purpose'].str.lower().str.contains(join)].index.to_list()
    for i in index:
        df.loc[i, 'purpose_category'] = category
    return df


categoryzer(wedding, 'свадьба')
categoryzer(estate, 'недвижимость')
categoryzer(auto, 'авто')
categoryzer(education, 'образование')

#проверим результат и убедимся, что значений None не осталось
Counter(df['purpose_category'])

Counter({'недвижимость': 10811,
         'авто': 4306,
         'образование': 4013,
         'свадьба': 2324})

**Вывод**

Видим, что многие уникальные значения цели кредита названы похожим образом, но имеют одинаковый смысл. Мы лемматизируем их, чтобы затем разбить на категории. Обе задачи мы выполнили с помощью одной функции. В итоге мы получили 4 категории целей кредита. В нашей задаче это не понадобится, но наша категоризация сможет помочь другим аналитикам, которые изучают влияние цели кредита на возврат денег в срок.

### Категоризация данных

In [20]:
debt_from_children = pd.DataFrame()
debt_from_children['count_children'] = df.groupby('children')['debt'].count() #посчитаем сколько имеют задолженность по кредиту
debt_from_children['sum_children'] = df.groupby('children')['debt'].sum() #посчитаем сумму задолженности по кредиту
debt_from_children['result_children'] = debt_from_children['sum_children'] / debt_from_children['count_children'] #посчитаем отношение суммы задолженности на количество кредитов

**Вывод**

Так как мы изучаем влияние семейного положения и наличия детей на факт погашения в срок, мы не будем разбивать возраста на категории, так как они нам не понадобятся. Для сдачи проекта категоризация была применена вместе с лемматизацией выше в одной функции. Категоризовать количество детей нет необходимости, категориями будет как раз их количество, так как всего 6 вариантов.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [21]:
debt_from_children = pd.DataFrame()
debt_from_children['count_children'] =  df.groupby('children')['debt'].count()
debt_from_children['sum_children'] = df.groupby('children')['debt'].sum()
debt_from_children['result_children'] = debt_from_children['sum_children'] / debt_from_children['count_children'] 
debt_from_children.sort_values('result_children', ascending = False) #отсортируем таблицу по столбцу 'result_children' и выведем на экран

,count_children,sum_children,result_children
children,,,
4,41,4,0.097561
2,2052,194,0.094542
1,4855,445,0.091658
3,330,27,0.081818
0,14167,1071,0.075598
5,9,0,0.000000


**Вывод**

По таблице видно, что семьи с тремя детьями чаще возвращают кредит, чем семьи с одним ребенком. Так же бездетные просрачивают реже, чем люди с детьями.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [22]:
debt_from_family_status = pd.DataFrame()
debt_from_family_status['sum_family_status'] = df.groupby('family_status')['debt'].sum()
debt_from_family_status['count_family_status'] = df.groupby('family_status')['debt'].count()
debt_from_family_status['result_family_status'] = debt_from_family_status['sum_family_status'] / debt_from_family_status['count_family_status'] 
debt_from_family_status.sort_values('result_family_status', ascending = False) #отсортируем таблицу по столбцу 'result_children' и выведем на экран

,sum_family_status,count_family_status,result_family_status
family_status,,,
Не женат / не замужем,274,2810,0.097509
гражданский брак,388,4151,0.093471
женат / замужем,931,12339,0.075452
в разводе,85,1195,0.071130
вдовец / вдова,63,959,0.065693


**Вывод**

По таблице видим, что люди не в браке и не бывавшие в браке, имеют больший процент по невозврату кредита, чем те кто развелись или овдовели.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [23]:
def total_income_category(value):
    if value >= 500000:
        return 10
    else:
        value = value // 50000 #выполним целочисленное деление, разбив людей на каждые 500000 дохода
        return value
df['total_income_id'] = df['total_income'].apply(total_income_category) #категоризуем уровень дохода создав доп столбец'total_income_id'
df['total_income_id'].value_counts()
debt_from_total_income = pd.DataFrame()
debt_from_total_income['sum'] = df.groupby('total_income_id')['debt'].sum()
debt_from_total_income['count'] = df.groupby('total_income_id')['debt'].count()
debt_from_total_income['conversion'] = debt_from_total_income['sum'] / debt_from_total_income['count'] #найдем конверсию зависимости дохода и возврата кредита
debt_from_total_income.sort_values('conversion', ascending = False) #отсортируем по столбцу

,sum,count,conversion
total_income_id,,,
2,624,7160,0.087151
3,405,4764,0.085013
6,51,624,0.081731
1,331,4091,0.080909
4,164,2254,0.072760
7,24,330,0.072727
8,13,196,0.066327
5,88,1330,0.066165
10,14,222,0.063063


**Вывод**

Зависимости между уровнем дохода и возвратом кредита в срок нет!

- Как разные цели кредита влияют на его возврат в срок?

In [24]:
debt_from_purpose_category = pd.DataFrame()
debt_from_purpose_category['sum_purpose_category'] = df.groupby('purpose_category')['debt'].sum()#посчитаем сумму всех целей в категории
debt_from_purpose_category['count_purpose_category'] = df.groupby('purpose_category')['debt'].count() #посчитаем сумму каждой строки целей
debt_from_purpose_category['result_purpose_category'] = debt_from_purpose_category['sum_purpose_category'] / debt_from_purpose_category['count_purpose_category'] #найдем отношение, оно точно покажет какие цели кредита влияют на его возврата в срок 
debt_from_purpose_category.sort_values('result_purpose_category', ascending = False)

,sum_purpose_category,count_purpose_category,result_purpose_category
purpose_category,,,
авто,403,4306,0.093590
образование,370,4013,0.092200
свадьба,186,2324,0.080034
недвижимость,782,10811,0.072334


**Вывод**

Как мы видим по таблице, за авто и образование чаще появляется задолженность, нежели за свадьбу и недвижимость

## Шаг 4. Общий вывод

Общий вывод - семейный статус влияет на вероятность платежей по кредиту в срок - люди, бывавшие в браке чаще плятят в срок чем те, кто не бывали в браке, причем разведенные и овдовевшие платят в срок чаще, чем люди в браке. Чем больше детей, тем чаще платят не в срок, бездетные реже просрачивают оплату.

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.